In [6]:
from sec_edgar_downloader import Downloader
import os
from bs4 import BeautifulSoup

In [21]:
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'AMD', 
           'BA', 'V', 'SPY', 'SPCE', 'BABA', 'MSTR', 'DIS', 'PYPL', 'SHOP', 'COIN', 
            'INTC', 'CSCO', 'IBM', 'GE', 'WMT', 'T']

In [22]:
dl = Downloader("NYU", "tan4742@nyu.edu", "../data/sec_data")

In [23]:
output_dir = "../data/transcripts"
os.makedirs(output_dir, exist_ok=True)

for ticker in tickers:
    print(f"Fetching 8-K filings for {ticker}...")
    
    # Download the latest 8-K filings
    dl.get("8-K", ticker, limit=5)
    
    # Path to downloaded files
    filing_dir = f"../data/sec_data/sec-edgar-filings/{ticker}/8-K/"
    
    # Find the latest filing
    if not os.path.exists(filing_dir):
        print(f"No filings found for {ticker}")
        continue
    
    for filing in os.listdir(filing_dir):
        filing_path = os.path.join(filing_dir, filing)
        for file in os.listdir(filing_path):
            if file.endswith(".htm") or file.endswith(".html"):
                with open(os.path.join(filing_path, file), "r", encoding="utf-8") as f:
                    soup = BeautifulSoup(f.read(), "html.parser")
                    
                    # Extract text from Exhibit 99 (where transcripts are usually found)
                    exhibits = soup.find_all("div")
                    text = "\n".join([ex.get_text() for ex in exhibits if "Exhibit 99" in ex.get_text()])
                    
                    if text:
                        output_file = os.path.join(output_dir, f"{ticker}_transcript.txt")
                        with open(output_file, "w", encoding="utf-8") as out:
                            out.write(text)
                        print(f"Transcript saved for {ticker}")

Fetching 8-K filings for AAPL...
Fetching 8-K filings for MSFT...
Fetching 8-K filings for GOOG...
Fetching 8-K filings for AMZN...
Fetching 8-K filings for TSLA...
Fetching 8-K filings for META...
Fetching 8-K filings for NFLX...
Fetching 8-K filings for NVDA...
Fetching 8-K filings for AMD...
Fetching 8-K filings for BA...
Fetching 8-K filings for V...
Fetching 8-K filings for SPY...
No filings found for SPY
Fetching 8-K filings for SPCE...
Fetching 8-K filings for BABA...
No filings found for BABA
Fetching 8-K filings for MSTR...
Fetching 8-K filings for DIS...
Fetching 8-K filings for PYPL...
Fetching 8-K filings for SHOP...
Fetching 8-K filings for COIN...
Fetching 8-K filings for INTC...
Fetching 8-K filings for CSCO...
Fetching 8-K filings for IBM...
Fetching 8-K filings for GE...
Fetching 8-K filings for WMT...
Fetching 8-K filings for T...
